In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_gru4rec_script(model_name, train_path, test_path, model_path, loss, optim, const_emb, embed, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, momentum, sample_alpha, bpreg, logq, hidden_act, n_epochs, n_sample, m):
    ps = f"layers={layers},loss={loss},bpreg={bpreg},logq={logq},final_act={final_act},hidden_act={hidden_act},n_epochs={n_epochs},batch_size={batch_size},dropout_p_embed={dropout_p_embed},dropout_p_hidden={dropout_p_hidden},learning_rate={learning_rate},momentum={momentum},sample_alpha={sample_alpha},n_sample={n_sample},constrained_embedding={const_emb},embedding={embed},adapt={optim},item_key=ItemId,session_key=SessionId,time_key=Time"
    s_train_full = f"THEANO_FLAGS=device=cuda0 python ./GRU4Rec/run.py {train_path} -s {model_path}/gru4rec/{model_name}.pickle -ps {ps} 2>&1"
    s_test_full = f"THEANO_FLAGS=device=cuda0 python ./GRU4Rec/run.py -t {test_path} -l ./trained_models/gru4rec/{model_name}.pickle -m {m} -e conservative"
    return s_train_full, s_test_full

In [ ]:
params = setups["coveo"]["params_bprmax"]
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test all features modell

In [ ]:
train_script_all, test_script_all = create_gru4rec_script(model_name='gru4rec_all_features', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=const_emb, embed=embed, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=momentum, sample_alpha=sample_alpha, bpreg=bpreg, logq=logq, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=n_sample, m=m)

In [ ]:
os.system(train_script_all)

In [ ]:
os.system(test_script_all)

## Train & test matching features with GRU4REC-pytorch

In [ ]:
train_script_matching1, test_script_matching1 = create_gru4rec_script(model_name='gru4rec_matching_gru4rec_pytorch', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=False, embed=layers, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=0.0, sample_alpha=0.0, bpreg=0.0, logq=0.0, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=0, m=m)

In [ ]:
os.system(train_script_matching1)

In [ ]:
os.system(test_script_matching1)

## Train & test matching features with Torch-GRU4Rec

In [ ]:
train_script_matching2, test_script_matching2 = create_gru4rec_script(model_name='gru4rec_matching_torch_gru4rec', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=False, embed=layers, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=0.0, sample_alpha=sample_alpha, bpreg=bpreg, logq=0.0, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=n_sample, m=m)

In [ ]:
os.system(train_script_matching2)

In [ ]:
os.system(test_script_matching2)


## Train & test matching features with Recpack

In [ ]:
train_script_matching3, test_script_matching3 = create_gru4rec_script(model_name='gru4rec_matching_recpack', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=False, embed=layers, final_act=final_act if loss!='bpr-max' else 'linear', layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=0.0, sample_alpha=0.0, bpreg=bpreg, logq=0.0, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=n_sample if loss=='bpr-max' else 0, m=m)

In [ ]:
os.system(train_script_matching3)

In [ ]:
os.system(test_script_matching3)

## Load the cross-entropy params (the tensorflow and keras models only support xe)

In [ ]:
params = setups["coveo"]["params_xe"]
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test matching features with GRU4Rec_TensorFlow

In [ ]:
train_script_matching4, test_script_matching4 = create_gru4rec_script(model_name='gru4rec_matching_gru4rec_tensorflow', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=False, embed=layers, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=0.0, sample_alpha=0.0, bpreg=0.0, logq=0.0, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=0, m=m)

In [ ]:
os.system(train_script_matching4)

In [ ]:
os.system(test_script_matching4)

## Train & test matching features with KerasGRU4Rec

In [ ]:
train_script_matching5, test_script_matching5 = create_gru4rec_script(model_name='gru4rec_matching_kerasgru4rec', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, const_emb=False, embed=0, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, momentum=0.0, sample_alpha=0.0, bpreg=0.0, logq=0.0, hidden_act=hidden_act, n_epochs=n_epochs, n_sample=0, m=m)

In [ ]:
os.system(train_script_matching5)

In [ ]:
os.system(test_script_matching5)